### exploratory data analysis

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
from sentence_transformers import SentenceTransformer, util
from sklearn.mixture import GaussianMixture
import sklearn as skl
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
# spark.stop()  # stop any already running 

/Users/karljaehnig/.virtualenvs/mlenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Read Cleaned Data") \
    .getOrCreate()


# Read the Parquet file into a DataFrame
s1clean = spark.read.parquet("online_retail_IIs1_clean.parquet/")
s2clean = spark.read.parquet("online_retail_IIs2_clean.parquet/")


24/05/20 10:02:29 WARN Utils: Your hostname, Karls-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.12.239 instead (on interface en0)
24/05/20 10:02:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/20 10:02:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### collect product description text

In [4]:
# Get unique product descriptions
unique_descriptions_s1 = s1clean.select("Description").distinct()
unique_descriptions_s2 = s2clean.select("Description").distinct()

# Combine unique descriptions from both sheets
unique_descriptions = unique_descriptions_s1.union(unique_descriptions_s2).distinct()

# Collect the unique descriptions to a list
unique_descriptions_list = [row['Description'] for row in unique_descriptions.collect()]
print(len(unique_descriptions_list))
print([unique_descriptions_list[ii] for ii in [random.randint(0,len(unique_descriptions_list)) for _ in range(10)]])

5189
['childrens spaceboy mug', 'brocade ring purse', 'gold mop orbit necklace', 'white chocolate scent candle', 'hook 5 hanger magic toadstool red', 'mini jigsaw circus parade', 'gold mug bone china tree of life', 'childrens circus parade mug', 'set of 16 vintage black cutlery', 'yellow flowers felt handbag kit']


### transform into semantic vector embeddings using sentence-transformers

In [5]:
# Load the pre-trained model and generate embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(unique_descriptions_list, show_progress_bar=True)
cosine_scores = util.cos_sim(embeddings, embeddings)


/Users/karljaehnig/.virtualenvs/mlenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|████████████████████████████████████████████████████| 163/163 [00:27<00:00,  6.01it/s]


In [26]:
print(cosine_scores.numpy())
with open("embeddings_cosine_distance_matrix.npy", 'wb') as f:
    np.save(f, cosine_scores)

with open("embeddings_cosine_distance_matrix.npy","rb") as f:
    placeholder = np.load(f)
    print(placeholder.shape)

[[ 1.          0.20998403  0.36944798 ...  0.34221828  0.04655176
   0.2100161 ]
 [ 0.20998403  1.          0.07578526 ...  0.342793    0.05265371
   0.06320201]
 [ 0.36944798  0.07578526  1.         ...  0.168783    0.1473761
   0.15418528]
 ...
 [ 0.34221822  0.34279302  0.16878298 ...  1.          0.08913241
   0.28104007]
 [ 0.04655177  0.05265371  0.14737612 ...  0.08913241  1.
  -0.02847959]
 [ 0.2100161   0.06320201  0.15418528 ...  0.28104007 -0.0284796
   1.0000001 ]]
(5189, 5189)


### cluster product desc vectors with DBSCAN

In [12]:
from sklearn.cluster import HDBSCAN


mdl = HDBSCAN(min_samples=1)
mdl.fit(cosine_scores)
print(np.unique(mdl.labels_).shape)
print(np.unique(mdl.labels_))
# Predict cluster labels for the descriptions


(343,)
[ -1   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232
 233 234 235 236 237 238 239 240 241 242 243

In [13]:
cosine_scores.to_cs
print(cosine_scores.shape)
from collections import Counter
Counter(mdl.labels_)

torch.Size([5189, 5189])


Counter({-1: 2029,
         340: 54,
         72: 51,
         161: 46,
         197: 36,
         57: 32,
         50: 32,
         332: 29,
         265: 28,
         268: 28,
         73: 26,
         204: 23,
         251: 22,
         213: 21,
         144: 21,
         327: 20,
         124: 20,
         107: 19,
         160: 19,
         123: 19,
         125: 19,
         206: 19,
         163: 19,
         299: 18,
         86: 18,
         341: 17,
         151: 17,
         51: 17,
         303: 17,
         284: 17,
         292: 17,
         95: 16,
         201: 16,
         188: 16,
         173: 15,
         296: 15,
         217: 15,
         205: 15,
         300: 15,
         104: 15,
         283: 14,
         80: 14,
         191: 14,
         143: 14,
         250: 14,
         103: 14,
         266: 13,
         67: 13,
         185: 13,
         156: 13,
         5: 13,
         128: 13,
         139: 13,
         192: 13,
         153: 13,
         237: 12,
  